# データセット

## カリフォルニア住宅価格
20,640件のカリフォルニアの住宅価格のデータが公開されているため、scikit-learn を利用して読み込む。

In [1]:
# ライブラリインポート
import pandas as pd
from sklearn.datasets import fetch_california_housing

# データ読み込み
california = fetch_california_housing()
california_df = pd.DataFrame(california.data)

# 列名を指定
california_df.columns = california.feature_names

# 目的変数を追加
california_df['MedHouseVal'] = california.target

In [2]:
# データ表示
california_df

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
0,8.3252,41.0,6.984127,1.023810,322.0,2.555556,37.88,-122.23,4.526
1,8.3014,21.0,6.238137,0.971880,2401.0,2.109842,37.86,-122.22,3.585
2,7.2574,52.0,8.288136,1.073446,496.0,2.802260,37.85,-122.24,3.521
3,5.6431,52.0,5.817352,1.073059,558.0,2.547945,37.85,-122.25,3.413
4,3.8462,52.0,6.281853,1.081081,565.0,2.181467,37.85,-122.25,3.422
...,...,...,...,...,...,...,...,...,...
20635,1.5603,25.0,5.045455,1.133333,845.0,2.560606,39.48,-121.09,0.781
20636,2.5568,18.0,6.114035,1.315789,356.0,3.122807,39.49,-121.21,0.771
20637,1.7000,17.0,5.205543,1.120092,1007.0,2.325635,39.43,-121.22,0.923
20638,1.8672,18.0,5.329513,1.171920,741.0,2.123209,39.43,-121.32,0.847


# モデル作成・アップロード

## 事前準備

In [9]:
# ライブラリインポート
import os
import warnings
import sys

import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.preprocessing import StandardScaler

import mlflow
import mlflow.sklearn

import logging

sys.path.append(os.path.join(os.getcwd(), '../../'))
from backend.elastic_manager.elastic_manager import ElasticManager

In [4]:
# mlflow設定
MLFLOW_SERVER_URI = os.environ["MLFLOW_SERVER_URI"]
MLFLOW_EXPERIMENT_NAME = os.environ["MLFLOW_EXPERIMENT_NAME"]

mlflow.set_tracking_uri(MLFLOW_SERVER_URI)
mlflow.set_experiment(MLFLOW_EXPERIMENT_NAME)

print(MLFLOW_SERVER_URI, MLFLOW_EXPERIMENT_NAME)

http://mlflow:5000 some


In [5]:
# 設定確認
print(mlflow.get_registry_uri())
print(mlflow.get_tracking_uri())
print(mlflow.list_experiments())

http://mlflow:5000
http://mlflow:5000
[<Experiment: artifact_location='s3://default/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>, <Experiment: artifact_location='s3://default/1', experiment_id='1', lifecycle_stage='active', name='some', tags={}>]


## モデル作成関数
train関数の引数に以下を指定して呼び出すことで、モデルの作成を行う。  
学習モデルやパラメーター、メトリックスをMLflowにアップロードする。  
  
- **df**： データ  
- **features**： 説明変数  
- **target**： 目的変数  
- **algorithm**： アルゴリズム  
- **parameter**： パラメーター  
  
モデルの結果は以下を指標に評価する。
  
- **RMSE**（Root Mean Squared Error）：二乗平均平方根誤差  
- **MAE**（Mean Absolute Error）：平均絶対誤差  
- **R2**（R squared）： 決定係数

In [6]:
def train(df, features, target, algorithm, parameter):
   
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)

    # モデル評価
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2


    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # データを学習用とテスト用に分割する
    train, test = train_test_split(df)

    # 説明変数と目的変数をそれぞれ指定
    train_x = train[features]
    test_x = test[features]
    train_y = train[[target]]
    test_y = test[[target]]

    # mlflow連携
    with mlflow.start_run(run_name=algorithm) as run:
        print(f"run_id: {run.info.run_id}")
        print(f"algorithm: {algorithm}")
        
        if algorithm == "LinearRegression":
            model = LinearRegression()
        elif algorithm == "Lasso":
            model = Lasso(alpha=parameter["alpha"])
        elif algorithm == "Ridge":
            model = Ridge(alpha=parameter["alpha"])
        elif algorithm == "ElasticNet":
            model = ElasticNet(alpha=parameter["alpha"], l1_ratio=parameter["l1_ratio"], random_state=42)
        elif algorithm == "SVR":
            model = SVR(kernel=parameter["kernel"], C=parameter["C"], epsilon=parameter["epsilon"], gamma=parameter["gamma"])
        
        # モデル学習
        model.fit(train_x, train_y)
        
        # テストデータによる予測
        predicted_qualities = model.predict(test_x)
        
        # 予測結果をもとにモデル評価
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # モデル評価結果を表示
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # 学習モデル・パラメーター・メトリックス（評価指標）をMLflowに保存
        for key, value in parameter.items():
            mlflow.log_param(key, value)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)

        mlflow.sklearn.log_model(model, "model")

## アルゴリズム一覧
以下から任意のアルゴリズムを選択し、アルゴリズムに応じてパラメーターを設定する。  
パラメーター、説明変数、目的変数を適宜変更して実行をする。
- **LinearRegression**  
パラメーター：なし
- **Lasso**  
パラメーター：alpha
- **Ridge**  
パラメーター：alpha
- **ElasticNet**  
パラメーター：alpha, l1_ratio

### LinearRegression
線形回帰。回帰によって連続値を予測する手法。

In [7]:
# 固定
df = california_df
algorithm = "LinearRegression"
parameter={}

# （※）以下は適宜変更

# 説明変数をリストとして指定
features = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"]

# 目的変数を指定
target = "MedHouseVal"

In [8]:
# 指定した値で関数を実行
train(df=df, features=features, target=target, algorithm=algorithm, parameter=parameter)

run_id: 2d8571ca724340f69cf39cb4e408c762
algorithm: LinearRegression
  RMSE: 0.7391493330191972
  MAE: 0.5411479553684728
  R2: 0.5982113822212163


### Ridge
線形回帰にL2正則化を行う手法。

In [10]:
# 固定
df = california_df
algorithm = "Ridge"

# （※）以下は適宜変更

# パラメーターを指定
parameter={"alpha": 1.0}

# 説明変数をリストとして指定
features = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"]

# 目的変数を指定
target = "MedHouseVal"

In [11]:
# 指定した値で関数を実行
train(df=df, features=features, target=target, algorithm=algorithm, parameter=parameter)

run_id: 7b6deb25605f4acbb0332c5faba70167
algorithm: Ridge
  RMSE: 0.739159316693047
  MAE: 0.5411601365857747
  R2: 0.5982005282477965


### Lasso
線形回帰にL1正則化を行う手法。

In [12]:
# 固定
df = california_df
algorithm = "Lasso"

# （※）以下は適宜変更

# パラメーターを指定
parameter={"alpha": 1.0}

# 説明変数をリストとして指定
features = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"]

# 目的変数を指定
target = "MedHouseVal"

In [13]:
# 指定した値で関数を実行
train(df=df, features=features, target=target, algorithm=algorithm, parameter=parameter)

run_id: 80b3e1354c8443a88bceed8411887483
algorithm: Lasso
  RMSE: 0.9887522759774892
  MAE: 0.7737455299786652
  R2: 0.2810341818007367


### ElasticNet
Ridge回帰によるL2正則化と，Lasso回帰によるL1正則化を合わせた手法。

In [15]:
# 固定
df = california_df
algorithm = "ElasticNet"

# （※）以下は適宜変更

# パラメーターを指定
parameter={"alpha": 0.5, "l1_ratio": 0.5}

# 説明変数をリストとして指定
features = ["MedInc", "HouseAge", "AveRooms", "AveBedrms", "Population", "AveOccup", "Latitude", "Longitude"]

# 目的変数を指定
target = "MedHouseVal"

In [16]:
# 指定した値で関数を実行
train(df=df, features=features, target=target, algorithm=algorithm, parameter=parameter)

run_id: 683b254c2c844eeca631a42cac368dda
algorithm: ElasticNet
  RMSE: 0.8405383940492157
  MAE: 0.6350501830593797
  R2: 0.48042486082617897
